In [1]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn import datasets, cross_validation, metrics, neighbors
from matplotlib.colors import ListedColormap
from pandas import DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/evgeny/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import pandas as pd
import xgboost


train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [3]:
train_items = list(train['item_id'])
test_items = list(test['item_id'])

In [4]:
dif_train_items = dict(zip(train_items, train_items)).values()
dif_test_items = dict(zip(test_items, test_items)).values()
dif_train_items = list(dif_train_items)
dif_test_items = list(dif_test_items)

In [5]:
print(len(dif_train_items))
print(len(dif_test_items))

286
227


In [6]:
for i in range(len(dif_train_items)):
    if (dif_train_items[i] not in dif_test_items):
        print(i, dif_train_items[i])

1 20442113
2 20442114
5 20442118
6 20442119
27 20438581
46 20442193
51 20442207
52 20442210
55 20442214
57 20442216
58 20442215
59 20442221
67 20438756
69 20448549
70 20443281
81 20439191
82 20439193
87 20440223
88 20440224
92 20438692
94 20440742
95 20438697
105 20427444
114 20437695
116 20438731
117 20440261
118 20440262
120 20438732
128 20441303
136 20438823
137 20433396
149 20442092
158 20441371
167 20443945
181 20427072
188 20439383
189 20439384
190 20439385
191 20439386
200 20441575
203 20441455
204 20441456
205 20441457
206 20441460
214 20441481
215 20441482
217 20438924
231 20447920
241 20439113
248 20438091
251 20438689
263 20440031
267 20441576
269 20442091
272 20441582
273 20441583
274 20441584
282 20448432
283 20443174


In [7]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean([(np.abs((y_true[i] - y_pred[i])) / (np.abs(y_true[i]) + np.abs(y_pred[i]))) 
                    for i in range(len(list(y_true)))]) * 200

In [8]:
frac = 1 # fraction of learning examples used for model fitting

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print (len(X), len(y), len(test))

72457 72457 2016


In [9]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.20, random_state=1
)

In [11]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [16]:
#%%time
from sklearn.ensemble import GradientBoostingRegressor
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=43)
model = xgboost.XGBRegressor(max_depth=20, n_estimators=78, learning_rate=1/12, reg_alpha=16,
                                 reg_lambda=3, min_child_weight=6)
model.fit(X_train, y_train, eval_metric='mae')

#preds = model.predict(test.drop(['Num'], axis=1))
preds = model.predict(X_test)
print(symmetric_mean_absolute_percentage_error(y_test, preds))

0.2 21.5131333168
0.4 21.4406841647
0.5 21.0727825774
1 21.069492806
3 20.7081438044
6 20.7389318521
10 20.8890405242
15 21.3909160053
16 20.9945071921
17 21.4348566156


In [54]:
print (len(preds))
print (len(test))

14492
2016


In [8]:
sample_submission['y'] = preds

In [9]:
sample_submission.head(5)

,Num,y
0,348622,2304.652344
1,348623,31646.498047
2,348624,320020.875000
3,348625,33232.246094
4,348626,28.298035


In [10]:
# In GBM you can get some negative predictions:
print (sample_submission[sample_submission['y'] < 0])

        Num         y
230  349523 -10.53048


In [11]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [12]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)